<a href="https://colab.research.google.com/github/virf96/Basico/blob/main/Perfilamiento_Cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examen 1

## Preparación de ambiente

### Carga de módulos

In [2]:
pip install geopandas

     |████████████████████████████████| 1.0MB 10.8MB/s 
     |████████████████████████████████| 6.5MB 46.3MB/s 
     |████████████████████████████████| 14.8MB 219kB/s 


In [6]:
pip install keplergl

     |████████████████████████████████| 7.9MB 7.1MB/s 
  Created wheel for keplergl: filename=keplergl-0.2.2-py2.py3-none-any.whl size=15942747 sha256=07d5d370ad6255fda48bc7441e22bac97ebedfadae62e70a5a932746435fdea6
  Stored in directory: /root/.cache/pip/wheels/0e/78/c4/04ea0dbf6447ad708393751cd6ea453fb682f4c1c734f2247b
Successfully built keplergl


In [7]:
import pandas as pd
import cufflinks as cf
import geopandas as gpd
from keplergl import KeplerGl
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering

cf.go_offline()
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 100)
pd.set_option('display.float_format', lambda x: "{:,.2f}".format(x))

### Funciones relevantes

In [8]:
def pivot_categories(df, cluster_column, categories):
    aux = df.copy()
    aux["dum"] = 1
    for i, col in enumerate(categories):
        agg_data = aux[[cluster_column, col, "dum"]].pivot_table(index = cluster_column, columns=col, aggfunc="sum", fill_value=0)
        agg_data.columns = [f"{col}_{x}" for x in agg_data.columns.droplevel(0)]
        for col in agg_data:
            agg_data[col] /= agg_data.sum(axis=1)
        if i == 0:
            final = agg_data.copy()
        else:
            final = final.merge(agg_data, left_index = True, right_index = True, how = "inner")
    return final

## Datos

### Lectura de sets

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
users = pd.read_csv("/content/drive/MyDrive/datasets/users.csv")
users_cuisine = pd.read_csv("/content/drive/MyDrive/datasets/usercuisine.csv")
rating = pd.read_csv("/content/drive/MyDrive/datasets/ratings.csv")
restaurants = pd.read_csv("/content/drive/MyDrive/datasets/restaurants.csv")
parking = pd.read_csv("/content/drive/MyDrive/datasets/parking.csv")
restaruants_cuisine = pd.read_csv("/content/drive/MyDrive/datasets/cuisine.csv")
user_payment = pd.read_csv("/content/drive/MyDrive/datasets/userpayment.csv")
restaurant_payment = pd.read_csv("/content/drive/MyDrive/datasets/payment_methods.csv")
hours = pd.read_csv("/content/drive/MyDrive/datasets/hours.csv")

In [11]:
rating.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userID          1161 non-null   object
 1   placeID         1161 non-null   int64 
 2   rating          1161 non-null   int64 
 3   food_rating     1161 non-null   int64 
 4   service_rating  1161 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 106.7 KB


### EDA

#### Usuarios

In [12]:
#Podemos notar 3 agrupaciones claras mediante la latitud y longitud de los puntos esto nos lleva a pensar que están en la misma geolocalización
users[["userID", "latitude", "longitude"]].set_index("userID").iplot(kind = "scatter", x="latitude", y = "longitude", mode = "markers")

In [38]:
# Observamos las ubicaciones de los elementos
map_1 = KeplerGl(height=500)
gdf = gpd.GeoDataFrame(users, geometry=gpd.points_from_xy(users.latitude, users.longitude))
map_1.add_data(data=gdf, name="Usuarios")
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Usuarios': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20…

#### Restaurantes

In [14]:
map_1 = KeplerGl(height=500)
gdf = gpd.GeoDataFrame(restaurants, geometry=gpd.points_from_xy(restaurants.latitude, restaurants.longitude))
map_1.add_data(data=gdf, name="Restaurantes")
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Restaurantes': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19…

#### Rating

In [39]:
rating.head()

userID  placeID  rating  food_rating  service_rating
0  U1077   135085       2            2               2
1  U1077   135038       2            2               1
2  U1077   132825       2            2               2
3  U1077   135060       1            2               2
4  U1068   135104       1            1               2

In [15]:
#Observamos que la mitad de la población califica de mala manera el servicio
#Tenemos 1,161,000 establecimientos
rating.describe()

placeID   rating  food_rating  service_rating
count   1,161.00 1,161.00     1,161.00        1,161.00
mean  134,192.04     1.20         1.22            1.09
std     1,100.92     0.77         0.79            0.79
min   132,560.00     0.00         0.00            0.00
25%   132,856.00     1.00         1.00            0.00
50%   135,030.00     1.00         1.00            1.00
75%   135,059.00     2.00         2.00            2.00
max   135,109.00     2.00         2.00            2.00

In [16]:
#Hay más personas que califican de buena manera el servicio
rating["rating"].value_counts()

2    486
1    421
0    254
Name: rating, dtype: int64

In [17]:
#Observamos las veces que calificaron el estblecimiento
# El promedio de rating que tiene
#El promedio de rating en comida
# El promedio de rating en servicio
rating.groupby("userID").agg({"placeID": "count", "rating": "mean", "food_rating": "mean", "service_rating": "mean", })

placeID  rating  food_rating  service_rating
userID                                              
U1001         9    1.11         1.22            1.22
U1002        10    1.40         1.40            1.00
U1003        13    1.62         1.69            1.46
U1004         8    1.88         1.88            1.75
U1005         9    1.33         1.44            1.00
U1006        11    1.36         1.18            1.64
U1007         9    0.89         1.11            0.67
U1008         9    1.11         1.22            0.89
U1009        11    1.36         1.27            1.09
U1010         8    0.88         1.12            0.88
U1011         3    1.33         1.33            0.67
U1012         5    1.80         1.80            2.00
U1013        10    1.50         1.50            0.70
U1014        11    1.36         1.00            1.09
U1015         8    0.75         0.88            1.12
U1016        13    1.92         2.00            1.69
U1017         3    1.00         0.67            1.00
U1018        10    0.80         0.80            0.90
U1019         9    0.00         0.00            0.00
U1020         4    1.50         1.75            1.00
U1021         3    2.00         2.00            1.33
U1022        14    1.29         1.21            1.43
U1023         4    0.50         0.50            0.50
U1024        15    1.20         1.33            0.80
U1025        11    1.55         1.18            1.18
U1026         5    1.00         1.40            1.00
U1027        11    0.91         0.91            0.91
U1028         7    1.00         1.14            1.00
U1029        10    0.90         0.90            0.70
U1030         8    0.62         0.50            0.62
U1031         3    0.00         0.00            0.00
U1032        10    1.00         1.00            0.30
U1033        11    1.73         1.00            0.91
U1034         6    0.50         0.33            0.50
U1035         6    1.67         1.33            1.83
U1036        12    1.75         1.83            1.75
U1037        11    1.09         1.00            0.82
U1038         5    0.80         1.60            1.60
U1039         3    1.33         1.33            1.67
U1040         4    1.25         1.50            1.25
U1041         6    1.67         1.83            1.50
U1042         5    1.60         2.00            1.20
U1043         5    1.00         1.20            1.00
U1044         5    1.40         1.60            1.40
U1045        10    1.20         1.40            1.30
U1046         9    1.33         1.44            1.33
U1047         3    0.00         0.00            0.00
U1048         7    1.57         1.57            1.29
U1049         9    0.00         0.00            0.00
U1050         7    0.00         1.00            0.14
U1051         4    1.00         1.25            1.25
U1052         4    1.50         1.25            1.25
U1053        13    1.77         1.54            1.23
U1054        11    1.91         1.73            1.27
U1055        10    1.90         2.00            2.00
U1056        10    1.80         1.80            1.80
U1057        11    0.91         0.55            0.45
U1058        10    1.60         1.50            1.20
U1059        10    1.60         1.30            1.20
U1060         4    1.75         1.75            1.75
U1061        18    1.61         1.83            1.67
U1062         9    0.00         0.00            0.00
U1063         5    1.40         1.40            1.20
U1064         5    1.00         0.60            1.40
U1065         5    1.20         0.80            1.20
U1066         5    1.20         1.60            1.20
U1067         6    1.00         0.83            1.00
U1068         8    0.62         0.62            0.62
U1069         9    0.11         0.22            0.00
U1070         3    1.33         1.33            0.67
U1071        13    1.85         1.85            1.92
U1072         4    0.75         1.00            1.00
U1073        11    0.00         0.00            0.00
U1074         3    2.00         1

In [18]:
#Observamos la en número de veces que fueron evaluados y el promedio de esa evaliaciones por establecimiento
rating.groupby("placeID").agg(["count", "mean"]).sort_values(by=("rating", "count"))

rating      food_rating      service_rating     
         count mean       count mean          count mean
placeID                                                 
135011       3 1.33           3 1.33              3 1.33
132668       3 1.00           3 1.00              3 1.00
134975       3 1.67           3 1.33              3 1.33
132766       3 0.67           3 1.00              3 0.67
135016       3 1.00           3 1.33              3 1.00
132717       3 1.33           3 1.33              3 1.00
135033       4 0.75           4 0.75              4 1.25
135018       4 1.50           4 1.75              4 1.50
135040       4 0.25           4 0.25              4 0.25
135013       4 1.75           4 2.00              4 1.75
135044       4 1.25           4 1.25              4 1.00
135000       4 1.25           4 1.00              4 1.00
134987       4 1.00           4 0.50              4 1.00
134976       4 1.75           4 1.75              4 1.00
135073       4 1.50           4 1.50              4 1.00
132937       4 1.50           4 1.00              4 1.00
132877       4 0.75           4 0.75              4 1.00
135074       4 1.75           4 1.75              4 1.75
132773       4 1.50           4 1.25              4 1.75
135035       4 1.50           4 1.50              4 1.50
132560       4 0.50           4 1.00              4 0.25
134992       4 1.00           4 1.50              4 1.00
135109       4 1.00           4 1.25              4 0.75
132626       4 1.25           4 1.00              4 1.00
132654       4 0.25           4 0.25              4 0.25
132583       4 1.00           4 1.00              4 1.25
132564       4 1.25           4 1.25              4 1.50
132706       4 0.75           4 1.25              4 1.00
132561       4 0.75           4 1.00              4 1.00
132667       4 1.25           4 2.00              4 1.25
132715       4 1.00           4 1.00              4 0.50
134983       5 1.00           5 1.00              5 1.20
132870       5 0.60           5 1.00              5 0.40
132885       5 0.60           5 0.60              5 0.40
132866       5 1.40           5 1.40              5 1.40
132665       5 0.80           5 0.80              5 1.00
135034       5 2.00           5 2.00              5 1.60
132858       5 0.80           5 1.40              5 0.60
132660       5 1.40           5 1.40              5 1.00
132594       5 0.60           5 1.20              5 0.60
135049       5 1.00           5 0.60              5 1.00
132955       5 2.00           5 1.80              5 1.80
132846       5 1.40           5 1.40              5 1.20
132845       5 0.80           5 0.60              5 0.80
132609       5 0.60           5 0.60              5 0.60
132925       5 1.00           5 1.20              5 1.20
132755       5 1.80           5 2.00              5 1.60
134999       5 1.60           5 1.60              5 1.60
132958       6 1.67           6 1.50              6 1.50
135088       6 1.00           6 1.17              6 1.00
132584       6 1.33           6 1.50              6 1.00
132630       6 1.17           6 1.17              6 1.00
132613       6 1.17           6 1.33              6 1.00
132608       6 1.00           6 1.17              6 0.83
132663       6 0.50           6 0.50              6 0.67
135080       6 1.00           6 1.00              6 1.00
132922       6 1.83           6 1.50              6 1.83
132767       6 1.17           6 1.33              6 1.17
132884       6 1.17           6 1.17              6 1.17
135019       6 1.33           6 1.50              6 0.83
132869       6 1.17           6 1.33              6 1.17
135048       6 1.00           6 1.33              6 1.33
132854       6 1.33           6 1.67              6 1.17
132847       6 1.17           6 1.33              6 0.67
135055       7 1.71           7 1.71              7 1.71
132861       7 1.43           7 1.57              7 1.43
132851       7 1.43           7 1.00              7 1.00
135001       7 1.4

#### User payment

In [19]:
#Distribución de medios de pago
user_payment["Upayment"].value_counts()

cash                   131
bank_debit_cards        22
VISA                    17
MasterCard-Eurocard      4
American_Express         3
Name: Upayment, dtype: int64

#### User cuisine

In [20]:
#Hay demasiadas etiquetas diferentes, bucaremos hacer representativas las menos representativas
users_cuisine["Rcuisine"].value_counts(True)

Mexican                0.29
American               0.03
Cafeteria              0.03
Pizzeria               0.03
Family                 0.02
Cafe-Coffee_Shop       0.02
Japanese               0.02
Italian                0.02
Hot_Dogs               0.02
Latin_American         0.02
Burgers                0.02
Chinese                0.02
Regional               0.02
Contemporary           0.02
Fast_Food              0.02
Afghan                 0.01
Breakfast-Brunch       0.01
Sushi                  0.01
Deli-Sandwiches        0.01
Moroccan               0.01
Bakery                 0.01
Tex-Mex                0.01
Bar                    0.01
Game                   0.01
Juice                  0.01
Diner                  0.01
Barbecue               0.01
Continental-European   0.01
Bagels                 0.01
Hawaiian               0.01
Soup                   0.01
Fusion                 0.01
Eastern_European       0.01
Dessert-Ice_Cream      0.01
Mongolian              0.01
Doughnuts           

In [21]:
users_cuisine["Rcuisine"] = users_cuisine["Rcuisine"].map(lambda x:x if x in ["Mexican", "American", "Cafeteria", "Pizzeria"] else "Otros")

In [22]:
#Distribución después de la agrupación
users_cuisine["Rcuisine"].value_counts(True)

Otros       0.62
Mexican     0.29
American    0.03
Pizzeria    0.03
Cafeteria   0.03
Name: Rcuisine, dtype: float64

#### Restaurant payment

In [23]:
#Distribución de los métodos de pago en los restaurantes
restaurant_payment["Rpayment"].str.lower().value_counts()

cash                   500
visa                   255
mastercard-eurocard    194
american_express       153
bank_debit_cards       130
diners_club             42
discover                11
checks                  10
gift_certificates        7
carte_blanche            7
japan_credit_bureau      5
Name: Rpayment, dtype: int64

#### Hours

In [26]:
#Horarios de apertura y cierre de los restaurates
hours

placeID         hours                  days  \
0      135111  00:00-23:30;  Mon;Tue;Wed;Thu;Fri;   
1      135111  00:00-23:30;                  Sat;   
2      135111  00:00-23:30;                  Sun;   
3      135110  08:00-19:00;  Mon;Tue;Wed;Thu;Fri;   
4      135110  00:00-00:00;                  Sat;   
...       ...           ...                   ...   
2334   132012  12:00-22:00;                  Sun;   
2335   132012  12:00-22:00;  Mon;Tue;Wed;Thu;Fri;   
2336   132012  12:00-22:00;  Mon;Tue;Wed;Thu;Fri;   
2337   132012  12:00-22:00;                  Sat;   
2338   132012  12:00-22:00;                  Sat;   

                                   full  
0     00:00-23:30; Mon;Tue;Wed;Thu;Fri;  
1                     00:00-23:30; Sat;  
2                     00:00-23:30; Sun;  
3     08:00-19:00; Mon;Tue;Wed;Thu;Fri;  
4                     00:00-00:00; Sat;  
...                                 ...  
2334                  12:00-22:00; Sun;  
2335  12:00-22:00; Mon;Tue;Wed;Thu;Fri;  
2336  12:00-22:00; Mon;Tue;Wed;Thu;Fri;  
2337                  12:00-22:00; Sat;  
2338                  12:00-22:00; Sat;  

[2339 rows x 4 columns]

In [25]:
hours["full"] = hours["hours"] + " " + hours["days"]

In [27]:
hours["full"].value_counts(True)

00:00-23:30; Sun;                   0.10
00:00-23:30; Sat;                   0.10
00:00-23:30; Mon;Tue;Wed;Thu;Fri;   0.09
00:00-00:00; Sun;                   0.02
00:00-00:00; Sat;                   0.01
                                    ... 
12:30-23:30; Sun;                   0.00
07:00-20:00; Sun;                   0.00
20:00-23:30; Sun;                   0.00
20:00-03:00; Sun;                   0.00
02:00-23:30; Sun;                   0.00
Name: full, Length: 613, dtype: float64

In [28]:
#Distriución de status de estacionamiento en los establecimientos
parking["parking_lot"].value_counts(True)

none                0.50
yes                 0.25
public              0.15
street              0.05
fee                 0.03
valet parking       0.03
validated parking   0.00
Name: parking_lot, dtype: float64

## Ecosistema analítico

### Sistema de recomendación

In [40]:
restaurants

placeID  latitude  longitude  \
0     134999     18.92     -99.18   
1     132825     22.15    -100.98   
2     135106     22.15    -100.98   
3     132667     23.75     -99.16   
4     132613     23.75     -99.17   
5     135040     22.14    -100.97   
6     132732     23.75     -99.17   
7     132875     22.15    -100.99   
8     132609     23.76     -99.17   
9     135082     22.15    -100.92   
10    135070     22.15    -100.92   
11    135069     22.14    -100.94   
12    135065     22.15    -101.00   
13    135076     22.18    -100.97   
14    135086     22.14    -101.01   
15    132870     22.14    -100.94   
16    132854     22.14    -100.94   
17    132937     22.15    -100.98   
18    132856     22.15    -100.97   
19    132668     23.74     -99.15   
20    135033     22.14    -100.97   
21    132862     22.15    -100.99   
22    134983     18.95     -99.24   
23    132851     22.14    -100.93   
24    135013     18.92     -99.17   
25    135042     22.16    -100.97   
26    132768     18.93     -99.23   
27    132561     23.73     -99.13   
28    135045     22.15    -100.98   
29    132847     22.14    -100.94   
30    135054     22.14    -100.92   
31    134975     18.94     -99.22   
32    135053     22.18    -101.01   
33    135050     22.17    -100.97   
34    135025     22.15    -100.98   
35    135030     22.15    -100.99   
36    135057     22.15    -100.96   
37    135079     22.16    -101.00   
38    134992     18.94     -99.25   
39    132954     22.16    -100.99   
40    135041     22.15    -100.98   
41    132608     23.76     -99.17   
42    135046     22.14    -101.00   
43    132564     23.73     -99.15   
44    132869     22.14    -100.92   
45    132925     22.15    -100.98   
46    132955     22.15    -101.01   
47    135059     22.15    -100.99   
48    135001     18.94     -99.24   
49    135063     22.16    -100.98   
50    135080     22.15    -101.00   
51    135000     18.87     -99.23   
52    135066     22.17    -100.97   
53    135034     22.14    -101.02   
54    135060     22.16    -100.98   
55    135055     22.15    -101.01   
56    135108     22.14    -100.93   
57    135074     22.15    -101.00   
58    135018     18.86     -99.22   
59    135064     22.15    -101.00   
60    135038     22.16    -100.98   
61    135081     22.16    -100.96   
62    132723     22.15    -101.02   
63    135051     22.15    -100.98   
64    135021     18.93     -99.22   
65    135052     22.15    -100.98   
66    132845     22.13    -100.90   
67    135011     18.91     -99.17   
68    132858     22.13    -100.94   
69    135026     22.15    -101.00   
70    132884     22.14    -101.03   
71    135016     18.87     -99.21   
72    135104     23.75     -99.17   
73    132630     23.75     -99.16   
74    135047     22.15    -100.99   
75    132951     22.15    -100.99   
76    132665     23.74     -99.13   
77    135044     22.14    -101.00   
78    135035     22.15    -101.02   
79    135071     22.13    -100.91   
80    132834     22.16    -100.99   
81    132846     22.14    -100.94   
82    135048     22.14    -101.00   
83    132717     23.73     -99.15   
84    132861     22.15    -101.02   
85    132885     22.18    -100.96   
86    132740     23.75     -99.17   
87    132830     22.15    -100.94   
88    132773     18.87     -99.21   
89    132706     23.73     -99.13   
90    132872     22.17    -100.99   
91    134987     18.93     -99.23   
92    132572     22.14    -100.99   
93    132958     22.14    -101.01   
94    132733     23.75     -99.16   
95    132922     22.15    -100.98   
96    132715     23.73     -99.16   
97    135027     22.15    -100.97   
98    132767     18.88     -99.16   
99    135039     22.15    -100.97   
100   134996     18.92     -99.22   
101   132755     22.15    -101.02   
102   135043     22.19    -100.94   
103   132654     23.74     -99.13   
104   135049     22.14    -101.03   
105   135073     22.15    -100.97   
106   132560     23.75     -99.17   
107   134

In [30]:
#GUardamos las variables continuas
ls_cont = ["latitude", "longitude"]
#GUardamos las variables categoricas
ls_disc = ["state", "alcohol", "smoking_area", "dress_code", "accessibility", "price", "Rambience", "franchise", "area", "other_services"]

In [41]:
restaurants[ls_cont].describe()

latitude  longitude
count    130.00     130.00
mean      21.86    -100.34
std        1.50       0.86
min       18.86    -101.03
25%       22.14    -100.99
50%       22.15    -100.96
75%       22.16     -99.22
max       23.76     -99.13

In [32]:
#Veamos un descriptivo de las distribuciones de las variables categoricas
for feat in ls_disc:
    display(restaurants[feat].value_counts(True).to_frame())

state
SLP               0.38
Morelos           0.15
?                 0.14
San Luis Potosi   0.11
tamaulipas        0.07
Tamaulipas        0.05
san luis potosi   0.03
slp               0.02
S.L.P.            0.02
mexico            0.02
morelos           0.01
s.l.p.            0.01
san luis potos    0.01

alcohol
No_Alcohol_Served     0.67
Wine-Beer             0.26
Full_Bar              0.07

smoking_area
none                   0.54
not permitted          0.19
section                0.18
permitted              0.07
only at bar            0.02

dress_code
informal        0.91
casual          0.08
formal          0.02

accessibility
no_accessibility           0.58
completely                 0.35
partially                  0.07

price
medium   0.46
low      0.35
high     0.19

Rambience
familiar       0.93
quiet          0.07

franchise
f       0.83
t       0.17

area
closed  0.88
open    0.12

other_services
none                0.92
variety             0.05
Internet            0.03

In [42]:
ls_cont = ["latitude", "longitude"]
ls_disc = ["alcohol", "smoking_area", "dress_code", "accessibility", "price", "Rambience", "franchise", "area", "other_services"]

In [43]:
restaruants_cuisine

placeID        Rcuisine  dum
0     135110         Spanish    1
1     135109         Italian    1
2     135107  Latin_American    1
3     135106         Mexican    1
4     135105       Fast_Food    1
..       ...             ...  ...
911   132005         Seafood    1
912   132004         Seafood    1
913   132003   International    1
914   132002         Seafood    1
915   132001   Dutch-Belgian    1

[916 rows x 3 columns]

In [44]:
#Agregamos un contador para hacer el  análisis descriptivo
restaruants_cuisine["dum"] = 1

In [45]:
#Observamos el peso de cada clase de comida
restaruants_cuisine["Rcuisine"].str.lower().value_counts(True)

mexican                0.26
international          0.07
american               0.06
dutch-belgian          0.06
italian                0.05
greek                  0.04
bar                    0.03
french                 0.03
cafe-coffee_shop       0.03
pizzeria               0.03
bar_pub_brewery        0.03
cafeteria              0.03
chinese                0.02
fast_food              0.02
seafood                0.02
japanese               0.02
german                 0.02
family                 0.02
burgers                0.01
mediterranean          0.01
vegetarian             0.01
contemporary           0.01
deli-sandwiches        0.01
steaks                 0.01
hot_dogs               0.01
latin_american         0.01
asian                  0.01
sushi                  0.01
bakery                 0.01
juice                  0.01
polish                 0.01
armenian               0.01
continental-european   0.00
dessert-ice_cream      0.00
southwestern           0.00
breakfast-brunch    

In [46]:
#Agrupamos para hcer más representativos los grupos
restaruants_cuisine["Rcuisine"] = restaruants_cuisine["Rcuisine"].map(lambda x:x if x in ["Mexican", "International", "American", "Dutch-Belgian", "Italian"] else "Otros")

In [47]:
#Contamos el número de veces en donde se repite alguna tipo de cocina en el establecimeinto esto normalmente solo pasa en otros
restaruants_cuisine = restaruants_cuisine.pivot_table(index = "placeID", columns="Rcuisine", aggfunc="sum", fill_value=0)

In [48]:
restaruants_cuisine

dum                                                  
Rcuisine American Dutch-Belgian International Italian Mexican Otros
placeID                                                            
132001          0             1             0       0       0     0
132002          0             0             0       0       0     1
132003          0             0             1       0       0     0
132004          0             0             0       0       0     1
132005          0             0             0       0       0     2
...           ...           ...           ...     ...     ...   ...
135105          0             0             0       0       0     1
135106          0             0             0       0       1     0
135107          0             0             0       0       0     1
135109          0             0             0       1       0     0
135110          0             0             0       0       0     1

[769 rows x 6 columns]

In [49]:
restaruants_cuisine.columns = restaruants_cuisine.columns.droplevel(0)

In [51]:
restaruants_cuisine.reset_index(inplace=True)

In [52]:
#Obtenemos una tabla tipo dummy
restaruants_cuisine

Rcuisine  placeID  American  Dutch-Belgian  International  Italian  Mexican  \
0          132001         0              1              0        0        0   
1          132002         0              0              0        0        0   
2          132003         0              0              1        0        0   
3          132004         0              0              0        0        0   
4          132005         0              0              0        0        0   
..            ...       ...            ...            ...      ...      ...   
764        135105         0              0              0        0        0   
765        135106         0              0              0        0        1   
766        135107         0              0              0        0        0   
767        135109         0              0              0        1        0   
768        135110         0              0              0        0        0   

Rcuisine  Otros  
0             0  
1             1  
2             0  
3             1  
4             2  
..          ...  
764           1  
765           0  
766           1  
767           0  
768           1  

[769 rows x 7 columns]

In [54]:
#Distribución de los métodos de pago
restaurant_payment["Rpayment"].str.lower().value_counts(True)

cash                  0.38
visa                  0.19
mastercard-eurocard   0.15
american_express      0.12
bank_debit_cards      0.10
diners_club           0.03
discover              0.01
checks                0.01
gift_certificates     0.01
carte_blanche         0.01
japan_credit_bureau   0.00
Name: Rpayment, dtype: float64

In [55]:
#Agrumos las categorías menos representativas de manera que haga más sentido al negocio
restaurant_payment["Rpayment"] = restaurant_payment["Rpayment"].str.lower().map(lambda x: x if x in ["cash", "visa", "mastercard-eurocard", "american_express", "bank_debit_cards"] else "otros")

In [56]:
#Agregamos un contador
restaurant_payment["dum"] = 1

In [57]:
restaurant_payment = restaurant_payment.pivot_table(index = "placeID", columns="Rpayment", aggfunc="sum", fill_value=0)

In [58]:
restaurant_payment

dum                                                     
Rpayment american_express bank_debit_cards cash mastercard-eurocard otros visa
placeID                                                                       
132002                  1                0    0                   1     1    1
132012                  1                1    0                   0     1    1
132019                  1                0    0                   1     1    1
132023                  1                1    0                   0     2    0
132024                  1                1    0                   0     1    1
...                   ...              ...  ...                 ...   ...  ...
135105                  0                0    1                   0     0    0
135106                  0                0    1                   1     0    1
135107                  1                1    1                   1     0    1
135109                  0                0    1                   0     0    0
135110                  1                1    1                   1     0    1

[615 rows x 6 columns]

In [59]:
restaurant_payment.columns = restaurant_payment.columns.droplevel(0)

In [60]:
restaurant_payment.reset_index(inplace=True)

In [61]:
#Obtenemos una tabla de conteo de métodos de pago para cada establecimiento
restaurant_payment

Rpayment  placeID  american_express  bank_debit_cards  cash  \
0          132002                 1                 0     0   
1          132012                 1                 1     0   
2          132019                 1                 0     0   
3          132023                 1                 1     0   
4          132024                 1                 1     0   
..            ...               ...               ...   ...   
610        135105                 0                 0     1   
611        135106                 0                 0     1   
612        135107                 1                 1     1   
613        135109                 0                 0     1   
614        135110                 1                 1     1   

Rpayment  mastercard-eurocard  otros  visa  
0                           1      1     1  
1                           0      1     1  
2                           1      1     1  
3                           0      2     0  
4                           0      1     1  
..                        ...    ...   ...  
610                         0      0     0  
611                         1      0     1  
612                         1      0     1  
613                         0      0     0  
614                         1      0     1  

[615 rows x 7 columns]

In [62]:
restaurant_rating = rating[["placeID", "rating", "food_rating", "service_rating"]].groupby("placeID").mean().reset_index()

In [64]:
# Observamos el promedio de ratings para cada establecimiento
restaurant_rating

placeID  rating  food_rating  service_rating
0     132560    0.50         1.00            0.25
1     132561    0.75         1.00            1.00
2     132564    1.25         1.25            1.50
3     132572    1.00         1.00            0.93
4     132583    1.00         1.00            1.25
5     132584    1.33         1.50            1.00
6     132594    0.60         1.20            0.60
7     132608    1.00         1.17            0.83
8     132609    0.60         0.60            0.60
9     132613    1.17         1.33            1.00
10    132626    1.25         1.00            1.00
11    132630    1.17         1.17            1.00
12    132654    0.25         0.25            0.25
13    132660    1.40         1.40            1.00
14    132663    0.50         0.50            0.67
15    132665    0.80         0.80            1.00
16    132667    1.25         2.00            1.25
17    132668    1.00         1.00            1.00
18    132706    0.75         1.25            1.00
19    132715    1.00         1.00            0.50
20    132717    1.33         1.33            1.00
21    132723    1.42         1.33            1.25
22    132732    0.62         0.88            1.00
23    132733    1.30         1.30            1.20
24    132740    0.75         1.25            0.62
25    132754    1.46         1.46            1.31
26    132755    1.80         2.00            1.60
27    132766    0.67         1.00            0.67
28    132767    1.17         1.33            1.17
29    132768    1.60         1.70            1.40
30    132773    1.50         1.25            1.75
31    132825    1.28         1.34            0.94
32    132830    0.92         1.17            0.92
33    132834    1.00         1.12            0.88
34    132845    0.80         0.60            0.80
35    132846    1.40         1.40            1.20
36    132847    1.17         1.33            0.67
37    132851    1.43         1.00            1.00
38    132854    1.33         1.67            1.17
39    132856    0.86         1.14            0.86
40    132858    0.80         1.40            0.60
41    132861    1.43         1.57            1.43
42    132862    1.39         1.39            1.44
43    132866    1.40         1.40            1.40
44    132869    1.17         1.33            1.17
45    132870    0.60         1.00            0.40
46    132872    0.92         1.17            1.17
47    132875    1.12         1.25            1.12
48    132877    0.75         0.75            1.00
49    132884    1.17         1.17            1.17
50    132885    0.60         0.60            0.40
51    132921    1.24         1.12            0.88
52    132922    1.83         1.50            1.83
53    132925    1.00         1.20            1.20
54    132937    1.50         1.00            1.00
55    132951    1.00         0.90            0.90
56    132954    1.00         0.78            0.89
57    132955    2.00         1.80            1.80
58    132958    1.67         1.50            1.50
59    134975    1.67         1.33            1.33
60    134976    1.75         1.75            1.00
61    134983    1.00         1.00            1.20
62    134986    2.00         2.00            2.00
63    134987    1.00         0.50            1.00
64    134992    1.00         1.50            1.00
65    134996    1.56         1.33            1.56
66    134999    1.60         1.60            1.60
67    135000    1.25         1.00            1.00
68    135001    1.43         1.43            1.57
69    135011    1.33         1.33            1.33
70    135013    1.75         2.00            1.75
71    135016    1.00         1.33            1.00
72    135018    1.50         1.75            1.50
73    135019    1.33         1.50            0.83
74    135021    1.33         1.11            0.67
75    135025    1.67         1.47            1.60
76    135026    1.36         1.18            1.45
77    135027    0.88         1.38            1.00
78    135028    1.53         1.40            1.27
79   

In [65]:
# Cruzamos las tablas para obtener las caracteristicas de los restaurantes y dado que necesitamos que las clases no se intersecten dummies es una buena opción
X_restaurant = restaurants[ls_cont+["placeID"]].join(pd.get_dummies(restaurants[ls_disc])).merge(restaruants_cuisine, on = "placeID", how = "inner").merge(restaurant_payment, on = "placeID", how = "inner").merge(restaurant_rating, on ="placeID", how = "inner")

In [81]:
X_restaurant.head(4)

latitude  longitude  placeID  alcohol_Full_Bar  alcohol_No_Alcohol_Served  \
0     22.15    -100.98   132825                 0                          1   
1     22.15    -100.98   135106                 0                          0   
2     23.75     -99.16   132667                 0                          1   
3     23.75     -99.17   132613                 0                          1   

   alcohol_Wine-Beer  smoking_area_none  smoking_area_not permitted  \
0                  0                  1                           0   
1                  1                  0                           0   
2                  0                  1                           0   
3                  0                  0                           0   

   smoking_area_only at bar  smoking_area_permitted  smoking_area_section  \
0                         0                       0                     0   
1                         1                       0                     0   
2                         0                       0                     0   
3                         0                       1                     0   

   dress_code_casual  dress_code_formal  dress_code_informal  \
0                  0                  0                    1   
1                  0                  0                    1   
2                  0                  0                    1   
3                  0                  0                    1   

   accessibility_completely  accessibility_no_accessibility  \
0                         1                               0   
1                         0                               0   
2                         1                               0   
3                         1                               0   

   accessibility_partially  price_high  price_low  price_medium  \
0                        0           0          1             0   
1                        1           0          0             1   
2                        0           0          1             0   
3                        0           0          0             1   

   Rambience_familiar  Rambience_quiet  franchise_f  franchise_t  area_closed  \
0                   1                0            1            0            0   
1                   1                0            1            0            0   
2                   1                0            0            1            1   
3                   1                0            0            1            1   

   area_open  other_services_Internet  other_services_none  \
0          1                        0                    1   
1          1                        0                    1   
2          0                        0                    1   
3          0                        0                    1   

   other_services_variety  American  Dutch-Belgian  International  Italian  \
0                       0         0              0              0        0   
1                       0         0              0              0        0   
2                       0         0              0              0        0   
3                       0         0              0              0        0   

   Mexican  Otros  american_express  bank_debit_cards  cash  \
0        1      0                 0                 0     1   
1        1      0                 0                 0     1   
2        0      1                 0                 0     1   
3        1      0                 0                 0     1   

   mastercard-eurocard  otros  visa  rating  food_rating  service_rating  \
0                    0      0     0    1.28         1.34            0.94   
1                    1      0     1    1.20         1.20            1.20   
2                    0      0     0    1.25         2.00            1.25   
3                    0      0     0    1.17         1.33            1.00   

                             name  
0                 puesto de tacos  
1      El 

In [67]:
#Mediante KNN podemos aproximar las caracteristicas de los restaurante para después recomendarlos
knn = NearestNeighbors(n_jobs=-1, n_neighbors=6, metric="euclidean")

In [68]:
knn.fit(X_restaurant[[x for x in X_restaurant if x != "placeID"]])

NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean',
                 metric_params=None, n_jobs=-1, n_neighbors=6, p=2, radius=1.0)

In [69]:
distances, indices = knn.kneighbors(X_restaurant[[x for x in X_restaurant if x != "placeID"]])

In [70]:
#Distancias más cercanas al cero
distances[0]

array([0.        , 2.08591412, 2.30386888, 2.48665983, 2.48945949,
       2.49534958])

In [71]:
#Indices de los elementos
indices[0]


array([ 0, 41, 52, 77, 57, 12])

In [82]:
recommend = X_restaurant.loc[[ 0, 41, 52, 77, 57, 12], ["placeID",'name']]

In [83]:
recommend

placeID                       name
0    132825            puesto de tacos
41   132858  Hamburguesas Valle Dorado
52   132885          Hamburguesas saul
77   132754            Cabana Huasteca
57   132572               Cafe Chaires
12   132856              Unicols Pizza

In [84]:
X_restaurant = X_restaurant.merge(restaurants[["name", "placeID"]], how = "left", on = "placeID")

In [85]:
restaurants

placeID  latitude  longitude  \
0     134999     18.92     -99.18   
1     132825     22.15    -100.98   
2     135106     22.15    -100.98   
3     132667     23.75     -99.16   
4     132613     23.75     -99.17   
5     135040     22.14    -100.97   
6     132732     23.75     -99.17   
7     132875     22.15    -100.99   
8     132609     23.76     -99.17   
9     135082     22.15    -100.92   
10    135070     22.15    -100.92   
11    135069     22.14    -100.94   
12    135065     22.15    -101.00   
13    135076     22.18    -100.97   
14    135086     22.14    -101.01   
15    132870     22.14    -100.94   
16    132854     22.14    -100.94   
17    132937     22.15    -100.98   
18    132856     22.15    -100.97   
19    132668     23.74     -99.15   
20    135033     22.14    -100.97   
21    132862     22.15    -100.99   
22    134983     18.95     -99.24   
23    132851     22.14    -100.93   
24    135013     18.92     -99.17   
25    135042     22.16    -100.97   
26    132768     18.93     -99.23   
27    132561     23.73     -99.13   
28    135045     22.15    -100.98   
29    132847     22.14    -100.94   
30    135054     22.14    -100.92   
31    134975     18.94     -99.22   
32    135053     22.18    -101.01   
33    135050     22.17    -100.97   
34    135025     22.15    -100.98   
35    135030     22.15    -100.99   
36    135057     22.15    -100.96   
37    135079     22.16    -101.00   
38    134992     18.94     -99.25   
39    132954     22.16    -100.99   
40    135041     22.15    -100.98   
41    132608     23.76     -99.17   
42    135046     22.14    -101.00   
43    132564     23.73     -99.15   
44    132869     22.14    -100.92   
45    132925     22.15    -100.98   
46    132955     22.15    -101.01   
47    135059     22.15    -100.99   
48    135001     18.94     -99.24   
49    135063     22.16    -100.98   
50    135080     22.15    -101.00   
51    135000     18.87     -99.23   
52    135066     22.17    -100.97   
53    135034     22.14    -101.02   
54    135060     22.16    -100.98   
55    135055     22.15    -101.01   
56    135108     22.14    -100.93   
57    135074     22.15    -101.00   
58    135018     18.86     -99.22   
59    135064     22.15    -101.00   
60    135038     22.16    -100.98   
61    135081     22.16    -100.96   
62    132723     22.15    -101.02   
63    135051     22.15    -100.98   
64    135021     18.93     -99.22   
65    135052     22.15    -100.98   
66    132845     22.13    -100.90   
67    135011     18.91     -99.17   
68    132858     22.13    -100.94   
69    135026     22.15    -101.00   
70    132884     22.14    -101.03   
71    135016     18.87     -99.21   
72    135104     23.75     -99.17   
73    132630     23.75     -99.16   
74    135047     22.15    -100.99   
75    132951     22.15    -100.99   
76    132665     23.74     -99.13   
77    135044     22.14    -101.00   
78    135035     22.15    -101.02   
79    135071     22.13    -100.91   
80    132834     22.16    -100.99   
81    132846     22.14    -100.94   
82    135048     22.14    -101.00   
83    132717     23.73     -99.15   
84    132861     22.15    -101.02   
85    132885     22.18    -100.96   
86    132740     23.75     -99.17   
87    132830     22.15    -100.94   
88    132773     18.87     -99.21   
89    132706     23.73     -99.13   
90    132872     22.17    -100.99   
91    134987     18.93     -99.23   
92    132572     22.14    -100.99   
93    132958     22.14    -101.01   
94    132733     23.75     -99.16   
95    132922     22.15    -100.98   
96    132715     23.73     -99.16   
97    135027     22.15    -100.97   
98    132767     18.88     -99.16   
99    135039     22.15    -100.97   
100   134996     18.92     -99.22   
101   132755     22.15    -101.02   
102   135043     22.19    -100.94   
103   132654     23.74     -99.13   
104   135049     22.14    -101.03   
105   135073     22.15    -100.97   
106   132560     23.75     -99.17   
107   134

In [86]:
X_restaurant

latitude  longitude  placeID  alcohol_Full_Bar  alcohol_No_Alcohol_Served  \
0      22.15    -100.98   132825                 0                          1   
1      22.15    -100.98   135106                 0                          0   
2      23.75     -99.16   132667                 0                          1   
3      23.75     -99.17   132613                 0                          1   
4      23.75     -99.17   132732                 0                          1   
5      22.15    -100.99   132875                 0                          0   
6      23.76     -99.17   132609                 0                          1   
7      22.14    -100.94   135069                 0                          0   
8      22.14    -101.01   135086                 0                          1   
9      22.14    -100.94   132870                 0                          1   
10     22.14    -100.94   132854                 0                          0   
11     22.15    -100.98   132937                 1                          0   
12     22.15    -100.97   132856                 0                          0   
13     22.15    -100.99   132862                 0                          0   
14     18.95     -99.24   134983                 1                          0   
15     22.14    -100.93   132851                 0                          1   
16     22.16    -100.97   135042                 0                          1   
17     18.93     -99.23   132768                 0                          1   
18     22.14    -100.92   135054                 0                          1   
19     22.18    -101.01   135053                 0                          1   
20     22.15    -100.98   135025                 0                          1   
21     22.15    -100.99   135030                 0                          1   
22     22.15    -100.96   135057                 0                          0   
23     22.16    -101.00   135079                 0                          1   
24     22.16    -100.99   132954                 0                          1   
25     22.15    -100.98   135041                 0                          0   
26     23.76     -99.17   132608                 0                          1   
27     22.14    -101.00   135046                 0                          1   
28     22.14    -100.92   132869                 0                          1   
29     22.15    -100.98   132925                 0                          0   
30     22.15    -101.01   132955                 0                          0   
31     22.15    -100.99   135059                 0                          0   
32     22.14    -101.02   135034                 0                          1   
33     22.16    -100.98   135060                 0                          1   
34     22.15    -101.01   135055                 0                          1   
35     22.15    -101.00   135074                 0                          0   
36     18.86     -99.22   135018                 1                          0   
37     22.15    -101.02   132723                 1                          0   
38     18.93     -99.22   135021                 0                          1   
39     22.15    -100.98   135052                 1                          0   
40     22.13    -100.90   132845                 0                          1   
41     22.13    -100.94   132858                 0                          1   
42     23.75     -99.17   135104                 1                          0   
43     22.15    -100.99   132951                 0                          1   
44     22.14    -101.00   135044                 0                          1   
45     22.15    -101.02   135035                 0                          1   
46     22.13    -100.91   135071                 1                          0   
47     22.16    -100.99   132834                 0                          1   
48     22.14    -100.94   132846   

In [87]:
recommend

placeID                       name
0    132825            puesto de tacos
41   132858  Hamburguesas Valle Dorado
52   132885          Hamburguesas saul
77   132754            Cabana Huasteca
57   132572               Cafe Chaires
12   132856              Unicols Pizza

In [111]:
for index, data in recommend.iterrows():
    print(f"Restaurants recommended for {data['name']}: ")
    for i, name in enumerate(X_restaurant.loc[indices[index], 'name_x']):
        if i == 0:
            continue
        print(f"\t{i}) {name}")

Restaurants recommended for puesto de tacos: 
	1) Hamburguesas Valle Dorado
	2) Hamburguesas saul
	3) Cabana Huasteca
	4) Cafe Chaires
	5) Unicols Pizza
Restaurants recommended for Hamburguesas Valle Dorado: 
	1) Hamburguesas saul
	2) Cafe Chaires
	3) KFC
	4) Unicols Pizza
	5) Tortas y hamburguesas el gordo
Restaurants recommended for Hamburguesas saul: 
	1) Hamburguesas Valle Dorado
	2) Cafe Chaires
	3) Abondance Restaurante Bar
	4) Tortas y hamburguesas el gordo
	5) KFC
Restaurants recommended for Cabana Huasteca: 
	1) Cenaduria El RincÃ³n de Tlaquepaque
	2) Restaurant Oriental Express
	3) Mariscos El Pescador
	4) Restaurant Wu Zhuo Yi
	5) Restaurante Marisco Sam
Restaurants recommended for Cafe Chaires: 
	1) KFC
	2) Hamburguesas Valle Dorado
	3) Tortas y hamburguesas el gordo
	4) Hamburguesas saul
	5) Chaires
Restaurants recommended for Unicols Pizza: 
	1) Hamburguesas Valle Dorado
	2) Abondance Restaurante Bar
	3) Hamburguesas saul
	4) Cafe Chaires
	5) el pueblito


### Clustering de usuarios

#### Ingeniería de datos

In [112]:
users

userID  latitude  longitude smoker     drink_level dress_preference  \
0    U1001     22.14    -100.98  false      abstemious         informal   
1    U1002     22.15    -100.98  false      abstemious         informal   
2    U1003     22.12    -100.95  false  social drinker           formal   
3    U1004     18.87     -99.18  false      abstemious         informal   
4    U1005     22.18    -100.96  false      abstemious    no preference   
5    U1006     22.15    -100.98   true  social drinker    no preference   
6    U1007     22.12    -100.94  false  casual drinker         informal   
7    U1008     22.12    -100.92  false  social drinker           formal   
8    U1009     22.16    -100.99  false      abstemious           formal   
9    U1010     22.19    -101.00  false  social drinker    no preference   
10   U1011     23.72     -99.15  false      abstemious    no preference   
11   U1012     18.81     -99.24  false  casual drinker           formal   
12   U1013     22.17    -100.99  false      abstemious    no preference   
13   U1014     23.75     -99.17  false      abstemious                ?   
14   U1015     22.13    -100.91   true  social drinker         informal   
15   U1016     22.16    -100.98  false  casual drinker         informal   
16   U1017     18.95     -99.20  false  casual drinker           formal   
17   U1018     22.19    -100.92   true  casual drinker           formal   
18   U1019     22.15    -100.98  false  casual drinker    no preference   
19   U1020     18.88     -99.22  false      abstemious    no preference   
20   U1021     23.73     -99.17  false  social drinker    no preference   
21   U1022     22.15    -100.96  false  casual drinker           formal   
22   U1023     23.75     -99.17  false  casual drinker         informal   
23   U1024     22.15    -100.98      ?      abstemious                ?   
24   U1025     22.13    -100.91  false  social drinker    no preference   
25   U1026     23.73     -99.13  false      abstemious           formal   
26   U1027     22.17    -100.99   true  social drinker    no preference   
27   U1028     23.75     -99.17  false      abstemious    no preference   
28   U1029     22.15    -100.99   true  casual drinker           formal   
29   U1030     18.84     -99.18  false  casual drinker           formal   
30   U1031     23.74     -99.16  false      abstemious         informal   
31   U1032     22.17    -100.99  false      abstemious           formal   
32   U1033     22.15    -100.98  false  social drinker    no preference   
33   U1034     22.14    -101.01  false  casual drinker         informal   
34   U1035     18.84     -99.22   true  casual drinker    no preference   
35   U1036     22.16    -100.99  false  social drinker         informal   
36   U1037     22.15    -100.90  false  social drinker    no preference   
37   U1038     22.13    -100.94   true  social drinker    no preference   
38   U1039     23.74     -99.14  false  social drinker    no preference   
39   U1040     18.90     -99.18  false      abstemious    no preference   
40   U1041     18.94     -99.24  false      abstemious    no preference   
41   U1042     18.93     -99.22  false      abstemious    no preference   
42   U1043     23.77     -99.17  false      abstemious    no preference   
43   U1044     18.95     -99.26   true  social drinker         informal   
44   U1045     22.16    -100.98  false  casual drinker          elegant   
45   U1046     22.14    -100.93  false  social drinker    no preference   
46   U1047     22.14    -100.95  false      abstemious    no preference   
47   U1048     22.14    -101.02   true  casual drinker    no preference   
48   U1049     22.15    -100.98  false  casual drinker         informal   
49   U1050     23.76     -99.17  false  social drinker    no preference   
50   U1051     18.88     -99.22  false  casual drinker    no preference   
51   U1052     22.14    -100.94   true  social drinker    no preference   
52   U1053     22.18    -100.

In [113]:
ls_disc_users = ["smoker", "drink_level", "dress_preference", "ambience", "transport", "marital_status", "hijos", "interest", "personality", "religion", "activity", "color", "budget"]

In [ ]:
for feat in ls_disc_users:
    display(users[feat].value_counts(True).to_frame())

smoker
false    0.79
true     0.19
?        0.02

drink_level
abstemious             0.37
casual drinker         0.34
social drinker         0.29

dress_preference
no preference              0.38
formal                     0.30
informal                   0.25
?                          0.04
elegant                    0.03

ambience
family        0.51
friends       0.33
solitary      0.12
?             0.04

transport
public          0.59
car owner       0.25
on foot         0.10
?               0.05

...

personality
hard-worker                 0.44
thrifty-protector           0.42
hunter-ostentatious         0.09
conformist                  0.05

religion
Catholic       0.72
none           0.22
Christian      0.05
Jewish         0.01
Mormon         0.01

activity
student            0.82
professional       0.11
?                  0.05
unemployed         0.01
working-class      0.01

color
blue     0.33
black    0.15
green    0.14
red      0.11
yellow   0.09
white    0.08
purple   0.08
orange   0.03

budget
medium    0.66
low       0.25
?         0.05
high      0.04

In [114]:
users_cuisine["dum"] = 1

In [115]:
users_cuisine = users_cuisine.pivot_table(index = "userID", columns = "Rcuisine", aggfunc="sum", fill_value=0)

In [116]:
users_cuisine.columns = users_cuisine.columns.droplevel(0)

In [117]:
users_cuisine = users_cuisine.reset_index()

In [118]:
users_cuisine

Rcuisine userID  American  Cafeteria  Mexican  Otros  Pizzeria
0         U1001         1          0        0      0         0
1         U1002         0          0        1      0         0
2         U1003         0          0        1      0         0
3         U1004         0          1        1      7         0
4         U1005         1          0        0      0         0
5         U1006         0          0        1      0         0
6         U1007         0          0        0      1         0
7         U1008         1          1        1      7         0
8         U1009         0          1        1      5         0
9         U1010         0          0        1      0         0
10        U1011         0          0        1      0         0
11        U1012         0          0        0      1         0
12        U1013         0          0        0      1         0
13        U1014         0          0        0      2         0
14        U1015         0          0        1      0         0
15        U1016         1          0        0     13         0
16        U1017         0          0        0      1         0
17        U1018         0          0        1      0         0
18        U1019         0          0        0      1         0
19        U1020         0          0        1      0         0
20        U1021         0          0        0      3         1
21        U1022         0          0        1      0         0
22        U1023         0          0        1      0         0
23        U1024         0          0        0      1         0
24        U1025         0          0        1      0         0
25        U1026         0          0        1      0         0
26        U1027         0          0        0      1         0
27        U1028         0          0        1      0         0
28        U1029         0          0        1      0         0
29        U1030         0          0        1      0         0
30        U1031         0          0        1      0         0
31        U1032         0          0        1      0         0
32        U1033         0          0        1      0         0
33        U1034         0          0        1      0         0
34        U1035         0          0        0      1         0
35        U1036         0          0        1      0         0
36        U1037         0          0        1      0         0
37        U1038         0          0        1      0         0
38        U1039         0          0        0      1         0
39        U1040         0          0        0      0         1
40        U1041         0          0        0      1         0
41        U1042         0          0        1      0         0
42        U1043         0          0        0      1         0
43        U1044         0          0        1      0         0
44        U1045         0          0        1      3         0
45        U1046         0          0        0      1         0
46        U1047         1          0        0      0         0
47        U1048         0          0        1      0         0
48        U1049         0          0        1      0         0
49        U1050         0          0        1      0         0
50        U1051         0          0        1      0         0
51        U1052         0          0        0      1         0
52        U1053         0          0        1      0         0
53        U1054         0          0        1      0         0
54        U1055         0          0        0      1         0
55        U1056         0          0        1      0         0
56        U1057         1          0        0      0         0
57        U1058         0          0        0      1         0
58        U1059         0          0        1      0         0
59        U1060         1          1        1      9         1
60        U1061         0          0        1      0         0
61        U1062         0          0        1      0         0
62        U1063         0      

In [119]:
user_payment["dum"] = 1

In [120]:
user_payment

userID             Upayment  dum
0    U1001                 cash    1
1    U1002                 cash    1
2    U1003                 cash    1
3    U1004                 cash    1
4    U1004     bank_debit_cards    1
5    U1005                 cash    1
6    U1006                 cash    1
7    U1007                 cash    1
8    U1008                 cash    1
9    U1009                 cash    1
10   U1010                 cash    1
11   U1011                 cash    1
12   U1012                 cash    1
13   U1012     bank_debit_cards    1
14   U1013  MasterCard-Eurocard    1
15   U1013                 cash    1
16   U1014                 cash    1
17   U1015                 cash    1
18   U1016                 cash    1
19   U1017                 cash    1
20   U1017                 VISA    1
21   U1018                 cash    1
22   U1019                 cash    1
23   U1020                 cash    1
24   U1020     bank_debit_cards    1
25   U1021                 cash    1
26   U1022                 cash    1
27   U1023                 cash    1
28   U1026                 cash    1
29   U1026     bank_debit_cards    1
30   U1027                 cash    1
31   U1028                 cash    1
32   U1029                 cash    1
33   U1030                 cash    1
34   U1031                 cash    1
35   U1031     American_Express    1
36   U1032                 cash    1
37   U1033                 cash    1
38   U1034                 cash    1
39   U1035                 cash    1
40   U1035                 VISA    1
41   U1036                 cash    1
42   U1037                 cash    1
43   U1038                 cash    1
44   U1039                 cash    1
45   U1040                 cash    1
46   U1041     bank_debit_cards    1
47   U1041                 cash    1
48   U1041     American_Express    1
49   U1041                 VISA    1
50   U1042     bank_debit_cards    1
51   U1043                 cash    1
52   U1044                 VISA    1
53   U1044     bank_debit_cards    1
54   U1044                 cash    1
55   U1044  MasterCard-Eurocard    1
56   U1045                 cash    1
57   U1046                 cash    1
58   U1047                 cash    1
59   U1048                 cash    1
60   U1049                 cash    1
61   U1050                 cash    1
62   U1051                 VISA    1
63   U1051                 cash    1
64   U1052                 cash    1
65   U1053                 cash    1
66   U1054                 cash    1
67   U1055                 cash    1
68   U1056                 cash    1
69   U1057                 cash    1
70   U1057                 VISA    1
71   U1058                 cash    1
72   U1059                 cash    1
73   U1060                 cash    1
74   U1061                 cash    1
75   U1061     bank_debit_cards    1
76   U1062                 VISA    1
77   U1062                 cash    1
78   U1063                 cash    1
79   U1064                 cash    1
80   U1065                 cash    1
81   U1066                 cash    1
82   U1066     bank_debit_cards    1
83   U1067                 cash    1
84   U1068                 cash    1
85   U1069  MasterCard-Eurocard    1
86   U1069                 cash    1
87   U1070                 cash    1
88   U1071                 cash    1
89   U1072     bank_debit_cards    1
90   U1072                 cash    1
91   U1073                 cash    1
92   U1074     bank_debit_cards    1
93   U1074                 cash    1
94   U1075                 cash    1
95   U1076                 VISA    1
96   U1076                 cash    1
97   U1076     bank_debit_cards    1
98   U1077                 VISA    1
99   U1077                 cash    1
100  U1077     bank_debit_cards    1
101  U1078                 VISA    1
102  U1078     bank_debit_cards    1
103  U1078                 cash    1
104  U1079                 cash    1
105  U1080                 cash    1
106  U1081                 cash    1
107  U10

In [121]:
user_payment = user_payment.pivot_table(index="userID", columns = "Upayment", aggfunc="sum", fill_value=0)

In [122]:
user_payment.columns = user_payment.columns.droplevel(0)

In [123]:
user_payment.reset_index(inplace=True)

In [124]:
user_payment.describe()

Upayment  American_Express  MasterCard-Eurocard   VISA  bank_debit_cards  \
count               133.00               133.00 133.00            133.00   
mean                  0.02                 0.03   0.13              0.17   
std                   0.15                 0.17   0.34              0.37   
min                   0.00                 0.00   0.00              0.00   
25%                   0.00                 0.00   0.00              0.00   
50%                   0.00                 0.00   0.00              0.00   
75%                   0.00                 0.00   0.00              0.00   
max                   1.00                 1.00   1.00              1.00   

Upayment   cash  
count    133.00  
mean       0.98  
std        0.12  
min        0.00  
25%        1.00  
50%        1.00  
75%        1.00  
max        1.00

In [126]:
user_rating = rating[["userID", "rating", "food_rating", "service_rating"]].groupby("userID").mean().reset_index()

In [127]:
user_rating

userID  rating  food_rating  service_rating
0    U1001    1.11         1.22            1.22
1    U1002    1.40         1.40            1.00
2    U1003    1.62         1.69            1.46
3    U1004    1.88         1.88            1.75
4    U1005    1.33         1.44            1.00
5    U1006    1.36         1.18            1.64
6    U1007    0.89         1.11            0.67
7    U1008    1.11         1.22            0.89
8    U1009    1.36         1.27            1.09
9    U1010    0.88         1.12            0.88
10   U1011    1.33         1.33            0.67
11   U1012    1.80         1.80            2.00
12   U1013    1.50         1.50            0.70
13   U1014    1.36         1.00            1.09
14   U1015    0.75         0.88            1.12
15   U1016    1.92         2.00            1.69
16   U1017    1.00         0.67            1.00
17   U1018    0.80         0.80            0.90
18   U1019    0.00         0.00            0.00
19   U1020    1.50         1.75            1.00
20   U1021    2.00         2.00            1.33
21   U1022    1.29         1.21            1.43
22   U1023    0.50         0.50            0.50
23   U1024    1.20         1.33            0.80
24   U1025    1.55         1.18            1.18
25   U1026    1.00         1.40            1.00
26   U1027    0.91         0.91            0.91
27   U1028    1.00         1.14            1.00
28   U1029    0.90         0.90            0.70
29   U1030    0.62         0.50            0.62
30   U1031    0.00         0.00            0.00
31   U1032    1.00         1.00            0.30
32   U1033    1.73         1.00            0.91
33   U1034    0.50         0.33            0.50
34   U1035    1.67         1.33            1.83
35   U1036    1.75         1.83            1.75
36   U1037    1.09         1.00            0.82
37   U1038    0.80         1.60            1.60
38   U1039    1.33         1.33            1.67
39   U1040    1.25         1.50            1.25
40   U1041    1.67         1.83            1.50
41   U1042    1.60         2.00            1.20
42   U1043    1.00         1.20            1.00
43   U1044    1.40         1.60            1.40
44   U1045    1.20         1.40            1.30
45   U1046    1.33         1.44            1.33
46   U1047    0.00         0.00            0.00
47   U1048    1.57         1.57            1.29
48   U1049    0.00         0.00            0.00
49   U1050    0.00         1.00            0.14
50   U1051    1.00         1.25            1.25
51   U1052    1.50         1.25            1.25
52   U1053    1.77         1.54            1.23
53   U1054    1.91         1.73            1.27
54   U1055    1.90         2.00            2.00
55   U1056    1.80         1.80            1.80
56   U1057    0.91         0.55            0.45
57   U1058    1.60         1.50            1.20
58   U1059    1.60         1.30            1.20
59   U1060    1.75         1.75            1.75
60   U1061    1.61         1.83            1.67
61   U1062    0.00         0.00            0.00
62   U1063    1.40         1.40            1.20
63   U1064    1.00         0.60            1.40
64   U1065    1.20         0.80            1.20
65   U1066    1.20         1.60            1.20
66   U1067    1.00         0.83            1.00
67   U1068    0.62         0.62            0.62
68   U1069    0.11         0.22            0.00
69   U1070    1.33         1.33            0.67
70   U1071    1.85         1.85            1.92
71   U1072    0.75         1.00            1.00
72   U1073    0.00         0.00            0.00
73   U1074    2.00         1.67            2.00
74   U1075    1.42         1.33            0.92
75   U1076    1.40         1.60            1.40
76   U1077    1.40         1.80            1.60
77   U1078    1.43         1.57            1.57
78   U1079    1.50         1.75            1.25
79   U1080    1.33         1.33            1.33
80   U1081    0.55         0.91            1.09
81   U1082    0.11         0.11            0.11
82   U1083    1.44  

#### TAD

In [128]:
X_users = users[["userID", "birth_year", "weight", "height"]].merge(users_cuisine, on = "userID", how = "outer").merge(user_payment, on = "userID", how = "outer").merge(user_rating, on = "userID", how = "outer").set_index("userID").fillna(0)

In [129]:
X_users.sample()

birth_year  weight  height  American  Cafeteria  Mexican  Otros  \
userID                                                                    
U1063         1988      80    1.78         0          0        1      0   

        Pizzeria  American_Express  MasterCard-Eurocard  VISA  \
userID                                                          
U1063          0              0.00                 0.00  0.00   

        bank_debit_cards  cash  rating  food_rating  service_rating  
userID                                                               
U1063               0.00  1.00    1.40         1.40            1.20

#### Escalamiento

In [130]:
sc = MinMaxScaler()

In [131]:
Xs = pd.DataFrame(data = sc.fit_transform(X_users), columns = X_users.columns, index = X_users.index)

In [132]:
Xs

birth_year  weight  height  American  Cafeteria  Mexican  Otros  \
userID                                                                    
U1001         0.92    0.36    0.71      1.00       0.00     0.00   0.00   
U1002         0.94    0.00    0.84      0.00       0.00     1.00   0.00   
U1003         0.92    0.25    0.61      0.00       0.00     1.00   0.00   
U1004         0.16    0.05    0.41      0.00       1.00     1.00   0.07   
U1005         0.97    0.31    0.61      1.00       0.00     0.00   0.00   
U1006         0.92    0.44    0.75      0.00       0.00     1.00   0.00   
U1007         0.92    0.25    0.49      0.00       0.00     0.00   0.01   
U1008         0.92    0.35    0.65      1.00       1.00     1.00   0.07   
U1009         0.95    0.44    0.73      0.00       1.00     1.00   0.05   
U1010         0.89    0.00    0.59      0.00       0.00     1.00   0.00   
U1011         0.92    0.35    0.73      0.00       0.00     1.00   0.00   
U1012         0.91    0.73    0.80      0.00       0.00     0.00   0.01   
U1013         0.81    0.50    0.69      0.00       0.00     0.00   0.01   
U1014         0.94    0.16    0.61      0.00       0.00     0.00   0.02   
U1015         0.92    0.59    0.59      0.00       0.00     1.00   0.00   
U1016         0.95    0.38    0.59      1.00       0.00     0.00   0.13   
U1017         0.95    0.49    0.74      0.00       0.00     0.00   0.01   
U1018         0.92    0.31    0.79      0.00       0.00     1.00   0.00   
U1019         0.92    0.24    0.50      0.00       0.00     0.00   0.01   
U1020         0.81    0.00    0.60      0.00       0.00     1.00   0.00   
U1021         0.84    0.85    0.53      0.00       0.00     0.00   0.03   
U1022         0.94    0.08    0.43      0.00       0.00     1.00   0.00   
U1023         0.91    0.79    0.74      0.00       0.00     1.00   0.00   
U1024         0.00    0.00    0.00      0.00       0.00     0.00   0.01   
U1025         0.94    0.31    0.74      0.00       0.00     1.00   0.00   
U1026         0.92    0.11    0.44      0.00       0.00     1.00   0.00   
U1027         0.92    0.25    0.56      0.00       0.00     0.00   0.01   
U1028         0.92    0.46    0.54      0.00       0.00     1.00   0.00   
U1029         0.94    0.23    0.50      0.00       0.00     1.00   0.00   
U1030         0.95    0.30    0.69      0.00       0.00     1.00   0.00   
U1031         0.97    0.00    0.50      0.00       0.00     1.00   0.00   
U1032         0.92    0.00    0.69      0.00       0.00     1.00   0.00   
U1033         0.95    0.18    0.50      0.00       0.00     1.00   0.00   
U1034         0.94    0.12    0.48      0.00       0.00     1.00   0.00   
U1035         0.86    0.36    0.69      0.00       0.00     0.00   0.01   
U1036         0.95    0.45    0.62      0.00       0.00     1.00   0.00   
U1037         0.94    0.15    0.38      0.00       0.00     1.00   0.00   
U1038         0.00    0.71    0.79      0.00       0.00     1.00   0.00   
U1039         0.92    0.54    0.66      0.00       0.00     0.00   0.01   
U1040         1.00    0.41    0.55      0.00       0.00     0.00   0.00   
U1041         0.86    0.00    0.53      0.00       0.00     0.00   0.01   
U1042         0.20    0.00    0.25      0.00       0.00     1.00   0.00   
U1043         0.92    0.25    0.55      0.00       0.00     0.00   0.01   
U1044         0.61    0.56    0.66      0.00       0.00     1.00   0.00   
U1045         0.95    0.35    0.50      0.00       0.00     1.00   0.03   
U1046         0.94    0.53    0.80      0.00       0.00     0.00   0.01   
U1047         0.97    0.25    0.54      1.00       0.00     0.00   0.00   
U1048         0.91    0.31    0.62      0.00       0.00     1.00   0.00   
U1049         0.95    0.19    0.50      0.00       0.00     1.00   0.00   
U1050         0.92    0.41    0.64      0.00       0.00     1.00   0.00   
U1051         0.77    0.43    0.70      0.00       0.00     1.00   0.00   
U1052         0.94    0.56    0.7

#### Visualización

In [133]:
mds = MDS(n_components=2)

In [134]:
Xm = pd.DataFrame(data=mds.fit_transform(Xs), columns=["d1", "d2"], index=X_users.index)

In [135]:
Xm.iplot(kind = "scatter", mode = "markers", x="d1", y = "d2")

#### Clustering

##### Dendograma

In [ ]:
Xs

birth_year  weight  height  American  Cafeteria  Mexican  Otros  \
userID                                                                    
U1001         0.92    0.36    0.71      1.00       0.00     0.00   0.00   
U1002         0.94    0.00    0.84      0.00       0.00     1.00   0.00   
U1003         0.92    0.25    0.61      0.00       0.00     1.00   0.00   
U1004         0.16    0.05    0.41      0.00       1.00     1.00   0.07   
U1005         0.97    0.31    0.61      1.00       0.00     0.00   0.00   
U1006         0.92    0.44    0.75      0.00       0.00     1.00   0.00   
U1007         0.92    0.25    0.49      0.00       0.00     0.00   0.01   
U1008         0.92    0.35    0.65      1.00       1.00     1.00   0.07   
U1009         0.95    0.44    0.73      0.00       1.00     1.00   0.05   
U1010         0.89    0.00    0.59      0.00       0.00     1.00   0.00   
U1011         0.92    0.35    0.73      0.00       0.00     1.00   0.00   
U1012         0.91    0.73    0.80      0.00       0.00     0.00   0.01   
U1013         0.81    0.50    0.69      0.00       0.00     0.00   0.01   
U1014         0.94    0.16    0.61      0.00       0.00     0.00   0.02   
U1015         0.92    0.59    0.59      0.00       0.00     1.00   0.00   
U1016         0.95    0.38    0.59      1.00       0.00     0.00   0.13   
U1017         0.95    0.49    0.74      0.00       0.00     0.00   0.01   
U1018         0.92    0.31    0.79      0.00       0.00     1.00   0.00   
U1019         0.92    0.24    0.50      0.00       0.00     0.00   0.01   
U1020         0.81    0.00    0.60      0.00       0.00     1.00   0.00   
U1021         0.84    0.85    0.53      0.00       0.00     0.00   0.03   
U1022         0.94    0.08    0.43      0.00       0.00     1.00   0.00   
U1023         0.91    0.79    0.74      0.00       0.00     1.00   0.00   
U1024         0.00    0.00    0.00      0.00       0.00     0.00   0.01   
U1025         0.94    0.31    0.74      0.00       0.00     1.00   0.00   
U1026         0.92    0.11    0.44      0.00       0.00     1.00   0.00   
U1027         0.92    0.25    0.56      0.00       0.00     0.00   0.01   
U1028         0.92    0.46    0.54      0.00       0.00     1.00   0.00   
U1029         0.94    0.23    0.50      0.00       0.00     1.00   0.00   
U1030         0.95    0.30    0.69      0.00       0.00     1.00   0.00   
U1031         0.97    0.00    0.50      0.00       0.00     1.00   0.00   
U1032         0.92    0.00    0.69      0.00       0.00     1.00   0.00   
U1033         0.95    0.18    0.50      0.00       0.00     1.00   0.00   
U1034         0.94    0.12    0.48      0.00       0.00     1.00   0.00   
U1035         0.86    0.36    0.69      0.00       0.00     0.00   0.01   
U1036         0.95    0.45    0.62      0.00       0.00     1.00   0.00   
U1037         0.94    0.15    0.38      0.00       0.00     1.00   0.00   
U1038         0.00    0.71    0.79      0.00       0.00     1.00   0.00   
U1039         0.92    0.54    0.66      0.00       0.00     0.00   0.01   
U1040         1.00    0.41    0.55      0.00       0.00     0.00   0.00   
U1041         0.86    0.00    0.53      0.00       0.00     0.00   0.01   
U1042         0.20    0.00    0.25      0.00       0.00     1.00   0.00   
U1043         0.92    0.25    0.55      0.00       0.00     0.00   0.01   
U1044         0.61    0.56    0.66      0.00       0.00     1.00   0.00   
U1045         0.95    0.35    0.50      0.00       0.00     1.00   0.03   
U1046         0.94    0.53    0.80      0.00       0.00     0.00   0.01   
U1047         0.97    0.25    0.54      1.00       0.00     0.00   0.00   
U1048         0.91    0.31    0.62      0.00       0.00     1.00   0.00   
U1049         0.95    0.19    0.50      0.00       0.00     1.00   0.00   
U1050         0.92    0.41    0.64      0.00       0.00     1.00   0.00   
U1051         0.77    0.43    0.70      0.00       0.00     1.00   0.00   
U1052         0.94    0.56    0.7

En los algoritmos no supervisados primero se debe probar unicamente con las variables númericas, después se pueden transformar las variables categoricas en caso de no etener mucha información cuidando que las categorías se intersecten es decir que no tengamos categoría unicas para determinados registros porque podriamos sesgar el modelo dado que empezaría a segmentar por esa diferencia única

Algunos consideran que hacer el dendograma con la reducción de dimensiones es erroneo, se puede probar las 2 y ver cuál va más en línea con el negocio

In [ ]:
dendrogram = sch.dendrogram(sch.linkage(Xs[[x for x in Xs.columns if x != "cl"]], method = 'ward'))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distance')
plt.show()

##### Clustering aglomerativo

In [ ]:
cluster = AgglomerativeClustering(linkage="ward", n_clusters=4)

##### Perfilamiento

In [ ]:
X_users["cl"] = Xs["cl"] = Xm["cl"] = [str(x) for x in cluster.fit_predict(Xs[[x for x in Xs.columns if x != "cl"]])]

In [ ]:
Xm.iplot(kind = "scatter", mode = "markers", x="d1", y = "d2", categories="cl")

/home/oscar/VirtualEnvs/Python 3/lib/python3.7/site-packages/cufflinks/plotlytools.py:807: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead

/home/oscar/VirtualEnvs/Python 3/lib/python3.7/site-packages/cufflinks/plotlytools.py:810: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [ ]:
X_users["cl"].value_counts(True)

1   0.43
2   0.22
0   0.17
3   0.17
Name: cl, dtype: float64

In [ ]:
aux = X_users[["cl"]].reset_index().merge(users[["userID"]+ls_disc_users], on ="userID", how = "inner")

In [ ]:
aux

userID cl smoker     drink_level dress_preference  ambience  transport  \
0    U1001  2  false      abstemious         informal    family    on foot   
1    U1002  1  false      abstemious         informal    family     public   
2    U1003  1  false  social drinker           formal    family     public   
3    U1004  3  false      abstemious         informal    family     public   
4    U1005  2  false      abstemious    no preference    family     public   
5    U1006  1   true  social drinker    no preference   friends  car owner   
6    U1007  2  false  casual drinker         informal  solitary     public   
7    U1008  0  false  social drinker           formal  solitary     public   
8    U1009  0  false      abstemious           formal    family    on foot   
9    U1010  1  false  social drinker    no preference   friends  car owner   
10   U1011  1  false      abstemious    no preference    family     public   
11   U1012  3  false  casual drinker           formal    family     public   
12   U1013  2  false      abstemious    no preference   friends     public   
13   U1014  2  false      abstemious                ?   friends     public   
14   U1015  1   true  social drinker         informal    family     public   
15   U1016  2  false  casual drinker         informal   friends    on foot   
16   U1017  2  false  casual drinker           formal    family     public   
17   U1018  1   true  casual drinker           formal   friends     public   
18   U1019  0  false  casual drinker    no preference    family     public   
19   U1020  3  false      abstemious    no preference   friends     public   
20   U1021  2  false  social drinker    no preference    family  car owner   
21   U1022  1  false  casual drinker           formal    family  car owner   
22   U1023  1  false  casual drinker         informal   friends  car owner   
23   U1024  1      ?      abstemious                ?         ?          ?   
24   U1025  1  false  social drinker    no preference    family     public   
25   U1026  3  false      abstemious           formal    family     public   
26   U1027  2   true  social drinker    no preference         ?     public   
27   U1028  1  false      abstemious    no preference    family     public   
28   U1029  1   true  casual drinker           formal    family     public   
29   U1030  0  false  casual drinker           formal    family    on foot   
30   U1031  0  false      abstemious         informal  solitary          ?   
31   U1032  1  false      abstemious           formal    family     public   
32   U1033  1  false  social drinker    no preference    family    on foot   
33   U1034  0  false  casual drinker         informal    family     public   
34   U1035  2   true  casual drinker    no preference   friends  car owner   
35   U1036  1  false  social drinker         informal    family     public   
36   U1037  1  false  social drinker    no preference    family     public   
37   U1038  1   true  social drinker    no preference    family     public   
38   U1039  2  false  social drinker    no preference   friends          ?   
39   U1040  2  false      abstemious    no preference   friends     public   
40   U1041  3  false      abstemious    no preference   friends    on foot   
41   U1042  3  false      abstemious    no preference    family  car owner   
42   U1043  2  false      abstemious    no preference  solitary     public   
43   U1044  3   true  social drinker         informal    family  car owner   
44   U1045  1  false  casual drinker          elegant   friends  car owner   
45   U1046  2  false  social drinker    no preference   friends  car owner   
46   U1047  2  false      abstemious    no preference    family     public   
47   U1048  1   true  casual drinker    no preference         ?     public   
48   U1049  0  false  casual drinker         informal  solitary     public   
49   U1050  0  false  social drinker    no preference    family  car owner   
50   U1051  2  false  casu

In [ ]:
X_users.groupby("cl").mean()

birth_year  weight  height  American  Cafeteria  Mexican  Otros  Pizzeria  \
cl                                                                              
0     1,989.04   61.83    1.68      0.21       0.33     0.83   6.25      0.21   
1     1,983.70   62.73    1.65      0.00       0.00     0.97   0.08      0.00   
2     1,987.55   72.81    1.71      0.19       0.00     0.10   1.13      0.13   
3     1,978.96   62.91    1.65      0.00       0.04     0.70   0.61      0.00   

    American_Express  MasterCard-Eurocard  VISA  bank_debit_cards  cash  \
cl                                                                        
0               0.04                 0.08  0.04              0.00  1.00   
1               0.00                 0.00  0.00              0.00  0.92   
2               0.00                 0.03  0.26              0.00  1.00   
3               0.09                 0.04  0.35              0.96  0.91   

    rating  food_rating  service_rating  
cl                                       
0     0.50         0.54            0.46  
1     1.34         1.32            1.22  
2     1.24         1.29            1.09  
3     1.46         1.60            1.42

In [ ]:
X_users.mean().to_frame().T

birth_year  weight  height  American  Cafeteria  Mexican  Otros  Pizzeria  \
0    1,984.70   64.87    1.67      0.08       0.07     0.70   1.48      0.07   

   American_Express  MasterCard-Eurocard  VISA  bank_debit_cards  cash  \
0              0.02                 0.03  0.12              0.16  0.95   

   rating  food_rating  service_rating  \
0    1.19         1.22            1.09   

                                                  cl  
0 1,531,313,044,298,704,436,140,209,303,001,540,7...

In [ ]:
pivot_categories(df=aux, cluster_column="cl", categories=ls_disc_users)

smoker_?  smoker_false  smoker_true  drink_level_abstemious  \
cl                                                                
0       0.00          0.88         0.77                    0.29   
1       0.05          0.79         0.93                    0.37   
2       0.00          0.77         0.90                    0.39   
3       0.00          0.83         0.83                    0.43   

    drink_level_casual drinker  drink_level_social drinker  \
cl                                                           
0                         0.75                        0.79   
1                         0.55                        0.95   
2                         0.41                        0.93   
3                         0.37                        0.91   

    dress_preference_?  dress_preference_elegant  dress_preference_formal  \
cl                                                                          
0                 0.00                      0.00                     0.29   
1                 0.07                      0.02                     0.44   
2                 0.03                      0.03                     0.21   
3                 0.00                      0.09                     0.19   

    dress_preference_informal  dress_preference_no preference  ambience_?  \
cl                                                                          
0                        0.52                            0.91        0.00   
1                        0.48                            0.94        0.08   
2                        0.26                            0.97        0.03   
3                        0.29                            0.95        0.00   

    ambience_family  ambience_friends  ambience_solitary  transport_?  \
cl                                                                      
0              0.46              0.59               0.83         0.04   
1              0.58              0.76               0.78         0.08   
2              0.43              0.74               0.77         0.03   
3              0.61              0.73               0.60         0.00   

    transport_car owner  transport_on foot  transport_public  \
cl                                                             
0                  0.17               0.21              0.97   
1                  0.24               0.12              0.99   
2                  0.27               0.18              0.97   
3                  0.43               0.07              0.96   

    marital_status_?  marital_status_married  marital_status_single  \
cl                                                                    
0               0.00                    0.04                   1.00   
1               0.07                    0.04                   1.00   
2               0.00                    0.06                   0.93   
3               0.00                    0.22                   0.99   

    marital_status_widow  hijos_?  hijos_dependent  hijos_independent  \
cl                                                                      
0                   0.00     0.08             0.05               0.85   
1                   0.00     0.12             0.00               0.96   
2                   0.67     0.03             0.07               0.96   
3                   0.00     0.04             0.00               0.77   

    hijos_kids  interest_eco-friendly  interest_none  interest_retro  \
cl                                                                     
0         0.75                   0.12           0.14            0.00   
1         0.65                   0.13           0.31            0.03   
2         0.49                   0.06           0.31            0.25   
3         0.86                   0.13           0.10            0.00   

    interest_technology  interest_variety  personality_conformist  \
cl                                                                  
0                  0.33              0.95

In [ ]:
X_users.groupby("cl").mean().merge(pivot_categories(df=aux, cluster_column="cl", categories=ls_disc_users), left_index=True, right_index=True)

birth_year  weight  height  American  Cafeteria  Mexican  Otros  Pizzeria  \
cl                                                                              
0     1,989.04   61.83    1.68      0.21       0.33     0.83   6.25      0.21   
1     1,983.70   62.73    1.65      0.00       0.00     0.97   0.08      0.00   
2     1,987.55   72.81    1.71      0.19       0.00     0.10   1.13      0.13   
3     1,978.96   62.91    1.65      0.00       0.04     0.70   0.61      0.00   

    American_Express  MasterCard-Eurocard  VISA  bank_debit_cards  cash  \
cl                                                                        
0               0.04                 0.08  0.04              0.00  1.00   
1               0.00                 0.00  0.00              0.00  0.92   
2               0.00                 0.03  0.26              0.00  1.00   
3               0.09                 0.04  0.35              0.96  0.91   

    rating  food_rating  service_rating  smoker_?  smoker_false  smoker_true  \
cl                                                                             
0     0.50         0.54            0.46      0.00          0.88         0.77   
1     1.34         1.32            1.22      0.05          0.79         0.93   
2     1.24         1.29            1.09      0.00          0.77         0.90   
3     1.46         1.60            1.42      0.00          0.83         0.83   

    drink_level_abstemious  drink_level_casual drinker  \
cl                                                       
0                     0.29                        0.75   
1                     0.37                        0.55   
2                     0.39                        0.41   
3                     0.43                        0.37   

    drink_level_social drinker  dress_preference_?  dress_preference_elegant  \
cl                                                                             
0                         0.79                0.00                      0.00   
1                         0.95                0.07                      0.02   
2                         0.93                0.03                      0.03   
3                         0.91                0.00                      0.09   

    dress_preference_formal  dress_preference_informal  \
cl                                                       
0                      0.29                       0.52   
1                      0.44                       0.48   
2                      0.21                       0.26   
3                      0.19                       0.29   

    dress_preference_no preference  ambience_?  ambience_family  \
cl                                                                
0                             0.91        0.00             0.46   
1                             0.94        0.08             0.58   
2                             0.97        0.03             0.43   
3                             0.95        0.00             0.61   

    ambience_friends  ambience_solitary  transport_?  transport_car owner  \
cl                                                                          
0               0.59               0.83         0.04                 0.17   
1               0.76               0.78         0.08                 0.24   
2               0.74               0.77         0.03                 0.27   
3               0.73               0.60         0.00                 0.43   

    transport_on foot  transport_public  marital_status_?  \
cl                                                          
0                0.21              0.97              0.00   
1                0.12              0.99              0.07   
2                0.18              0.97              0.00   
3                0.07              0.96              0.00   

    marital_status_married  marital_status_single  marital_status_widow  \
cl                                                                        
0                     0

In [ ]:
X_users.groupby("cl").mean().merge(pivot_categories(df=aux, cluster_column="cl", categories=ls_disc_users), left_index=True, right_index=True).style.background_gradient(cmap='Blues')

### Clustering de restaurantes

#### TAD

In [ ]:
X_restaurant = restaurant_payment.merge(restaurant_rating, on ="placeID", how = "outer").merge(restaruants_cuisine, on = "placeID", how = "outer").merge(restaurants[["placeID"]], on = "placeID", how = "right").fillna(0).set_index("placeID")

In [ ]:
X_restaurant

american_express  bank_debit_cards  cash  mastercard-eurocard  otros  \
placeID                                                                         
134999               0.00              0.00  0.00                 0.00   0.00   
132825               0.00              0.00  1.00                 0.00   0.00   
135106               0.00              0.00  1.00                 1.00   0.00   
132667               0.00              0.00  1.00                 0.00   0.00   
132613               0.00              0.00  1.00                 0.00   0.00   
135040               0.00              0.00  1.00                 1.00   0.00   
132732               0.00              0.00  1.00                 0.00   0.00   
132875               1.00              1.00  1.00                 1.00   0.00   
132609               0.00              0.00  1.00                 0.00   0.00   
135082               0.00              0.00  1.00                 0.00   0.00   
135070               0.00              0.00  1.00                 0.00   0.00   
135069               0.00              0.00  1.00                 0.00   0.00   
135065               1.00              0.00  1.00                 1.00   0.00   
135076               1.00              0.00  1.00                 1.00   0.00   
135086               0.00              0.00  1.00                 1.00   0.00   
132870               0.00              0.00  1.00                 0.00   0.00   
132854               0.00              1.00  1.00                 0.00   0.00   
132937               0.00              0.00  1.00                 0.00   0.00   
132856               0.00              0.00  1.00                 0.00   0.00   
132668               0.00              0.00  0.00                 0.00   0.00   
135033               1.00              0.00  1.00                 1.00   0.00   
132862               1.00              1.00  1.00                 1.00   0.00   
134983               0.00              0.00  1.00                 1.00   0.00   
132851               0.00              0.00  1.00                 0.00   0.00   
135013               0.00              0.00  1.00                 0.00   0.00   
135042               0.00              0.00  1.00                 0.00   0.00   
132768               0.00              1.00  1.00                 0.00   0.00   
132561               0.00              0.00  0.00                 0.00   0.00   
135045               1.00              0.00  1.00                 1.00   0.00   
132847               0.00              0.00  1.00                 0.00   0.00   
135054               0.00              0.00  1.00                 1.00   0.00   
134975               0.00              0.00  1.00                 1.00   1.00   
135053               1.00              0.00  1.00                 1.00   0.00   
135050               1.00              0.00  1.00                 1.00   0.00   
135025               1.00              0.00  1.00                 1.00   0.00   
135030               0.00              0.00  1.00                 0.00   0.00   
135057               1.00              0.00  1.00                 1.00   0.00   
135079               1.00              0.00  1.00                 0.00   0.00   
134992               0.00              0.00  1.00                 1.00   0.00   
132954               0.00              1.00  1.00                 1.00   0.00   
135041               0.00              0.00  1.00                 0.00   0.00   
132608               0.00              0.00  1.00                 0.00   0.00   
135046               0.00              0.00  1.00                 1.00   0.00   
132564               0.00              0.00  0.00                 0.00   0.00   
132869               0.00              0.00  1.00                 0.00   0.00   
132925               0.00              0.00  1.00                 0.00   0.00   
132955               0.00              1.00  1.00                 1.00   1.00   
135059               0.00              0

#### Clustering

##### Escalamiento

In [ ]:
sc = MinMaxScaler()

In [ ]:
Xs = pd.DataFrame(columns=X_restaurant.columns, index=X_restaurant.index, data = sc.fit_transform(X_restaurant))

##### Visualización

In [ ]:
mds = MDS(n_components=2)

In [ ]:
Xm = pd.DataFrame(columns=["d1", "d2"], data=mds.fit_transform(Xs), index = X_restaurant.index)

In [ ]:
Xm.iplot(kind="scatter", mode="markers", x="d1", y="d2")

##### Clustering jerárquico aglomerativo

In [ ]:
dendrogram = sch.dendrogram(sch.linkage(Xs, method = 'ward'))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distance')
plt.show()

In [ ]:
cluster = AgglomerativeClustering(n_clusters=2, linkage="ward")

##### Perfilamiento

In [ ]:
X_restaurant["cl"] = Xs["cl"] = Xm["cl"] = [str(x) for x in cluster.fit_predict(Xs)]

In [ ]:
aux = X_restaurant[["cl"]].reset_index().merge(restaurants[["placeID"]+ls_disc], on ="placeID", how = "inner")

In [ ]:
Xm.iplot(kind="scatter", mode="markers", x="d1", y="d2", categories="cl")

/home/oscar/VirtualEnvs/Python 3/lib/python3.7/site-packages/cufflinks/plotlytools.py:807: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead

/home/oscar/VirtualEnvs/Python 3/lib/python3.7/site-packages/cufflinks/plotlytools.py:810: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [ ]:
X_restaurant.groupby("cl").mean().style.background_gradient(cmap='Blues')

In [ ]:
X_restaurant.mean().to_frame().T

american_express  bank_debit_cards  cash  mastercard-eurocard  otros  visa  \
0              0.21              0.10  0.87                 0.35   0.02  0.39   

   rating  food_rating  service_rating  American  Dutch-Belgian  \
0    1.18         1.21            1.09      0.04           0.00   

   International  Italian  Mexican  Otros  \
0           0.03     0.03     0.22   0.55   

                                                  cl  
0 77,000,769,239,307,693,493,199,250,802,625,083,...

In [ ]:
X_restaurant.groupby("cl").mean().merge(pivot_categories(df=aux, categories=ls_disc, cluster_column="cl"), left_index = True, right_index=True).style.background_gradient(cmap='Blues')

### Solución compuesta

In [ ]:
cl_users = X_users.reset_index()[["userID", "cl"]].rename(columns={"cl": "cl_user"})

In [ ]:
cl_users

userID cl_user
0    U1001       2
1    U1002       1
2    U1003       1
3    U1004       3
4    U1005       2
5    U1006       1
6    U1007       2
7    U1008       0
8    U1009       0
9    U1010       1
10   U1011       1
11   U1012       3
12   U1013       2
13   U1014       2
14   U1015       1
15   U1016       2
16   U1017       2
17   U1018       1
18   U1019       0
19   U1020       3
20   U1021       2
21   U1022       1
22   U1023       1
23   U1024       1
24   U1025       1
25   U1026       3
26   U1027       2
27   U1028       1
28   U1029       1
29   U1030       0
30   U1031       0
31   U1032       1
32   U1033       1
33   U1034       0
34   U1035       2
35   U1036       1
36   U1037       1
37   U1038       1
38   U1039       2
39   U1040       2
40   U1041       3
41   U1042       3
42   U1043       2
43   U1044       3
44   U1045       1
45   U1046       2
46   U1047       2
47   U1048       1
48   U1049       0
49   U1050       0
50   U1051       2
51   U1052       2
52   U1053       1
53   U1054       1
54   U1055       2
55   U1056       1
56   U1057       2
57   U1058       2
58   U1059       1
59   U1060       0
60   U1061       3
61   U1062       2
62   U1063       1
63   U1064       1
64   U1065       1
65   U1066       3
66   U1067       1
67   U1068       0
68   U1069       0
69   U1070       1
70   U1071       1
71   U1072       3
72   U1073       2
73   U1074       3
74   U1075       1
75   U1076       3
76   U1077       3
77   U1078       3
78   U1079       1
79   U1080       1
80   U1081       1
81   U1082       0
82   U1083       1
83   U1084       1
84   U1085       1
85   U1086       3
86   U1087       2
87   U1088       1
88   U1089       1
89   U1090       2
90   U1091       0
91   U1092       0
92   U1093       3
93   U1094       0
94   U1095       2
95   U1096       1
96   U1097       1
97   U1098       1
98   U1099       2
99   U1100       1
100  U1101       0
101  U1102       2
102  U1103       3
103  U1104       2
104  U1105       0
105  U1106       3
106  U1107       3
107  U1108       0
108  U1109       1
109  U1110       1
110  U1111       1
111  U1112       0
112  U1113       2
113  U1114       0
114  U1115       2
115  U1116       3
116  U1117       3
117  U1118       0
118  U1119       1
119  U1120       1
120  U1121       3
121  U1122       1
122  U1123       1
123  U1124       1
124  U1125       1
125  U1126       1
126  U1127       1
127  U1128       0
128  U1129       0
129  U1130       1
130  U1131       1
131  U1132       1
132  U1133       3
133  U1134       1
134  U1135       0
135  U1136       1
136  U1137       1
137  U1138       2

In [ ]:
cl_restaurants = X_restaurant.reset_index()[["placeID", "cl"]].rename(columns={"cl": "cl_restaurants"})

In [ ]:
cl_restaurants

placeID cl_restaurants
0     134999              0
1     132825              0
2     135106              1
3     132667              0
4     132613              0
5     135040              1
6     132732              0
7     132875              1
8     132609              0
9     135082              0
10    135070              0
11    135069              0
12    135065              1
13    135076              1
14    135086              1
15    132870              0
16    132854              0
17    132937              0
18    132856              0
19    132668              0
20    135033              1
21    132862              1
22    134983              1
23    132851              0
24    135013              0
25    135042              0
26    132768              0
27    132561              0
28    135045              1
29    132847              0
30    135054              1
31    134975              1
32    135053              1
33    135050              1
34    135025              1
35    135030              0
36    135057              1
37    135079              0
38    134992              1
39    132954              1
40    135041              0
41    132608              0
42    135046              1
43    132564              0
44    132869              0
45    132925              0
46    132955              1
47    135059              0
48    135001              1
49    135063              0
50    135080              1
51    135000              0
52    135066              1
53    135034              1
54    135060              0
55    135055              1
56    135108              0
57    135074              1
58    135018              1
59    135064              1
60    135038              0
61    135081              1
62    132723              1
63    135051              0
64    135021              0
65    135052              1
66    132845              0
67    135011              0
68    132858              0
69    135026              0
70    132884              0
71    135016              0
72    135104              1
73    132630              0
74    135047              1
75    132951              0
76    132665              0
77    135044              0
78    135035              1
79    135071              1
80    132834              0
81    132846              0
82    135048              1
83    132717              0
84    132861              1
85    132885              0
86    132740              0
87    132830              0
88    132773              0
89    132706              1
90    132872              0
91    134987              0
92    132572              0
93    132958              1
94    132733              1
95    132922              0
96    132715              0
97    135027              1
98    132767              0
99    135039              0
100   134996              0
101   132755              1
102   135043              0
103   132654              0
104   135049              0
105   135073              1
106   132560              0
107   134976              1
108   135075              0
109   132660              0
110   132921              0
111   132594              0
112   134986              0
113   135062              0
114   132766              0
115   135088              0
116   135058              1
117   132754              0
118   132584              0
119   135032              1
120   135028              1
121   135085              0
122   132663              0
123   132626              0
124   132583              1
125   132866              0
126   135072              1
127   135109              0
128   135019              0
129   132877              0

In [ ]:
rating

userID  placeID  rating  food_rating  service_rating
0     U1077   135085       2            2               2
1     U1077   135038       2            2               1
2     U1077   132825       2            2               2
3     U1077   135060       1            2               2
4     U1068   135104       1            1               2
...     ...      ...     ...          ...             ...
1156  U1043   132630       1            1               1
1157  U1011   132715       1            1               0
1158  U1068   132733       1            1               0
1159  U1068   132594       1            1               1
1160  U1068   132660       0            0               0

[1161 rows x 5 columns]

In [ ]:
cl_ratings = rating.merge(cl_users, on ="userID", how = "inner").merge(cl_restaurants, on ="placeID", how ="inner")

In [ ]:
cl_ratings

userID  placeID  rating  food_rating  service_rating cl_user  \
0     U1077   135085       2            2               2       3   
1     U1108   135085       1            2               1       0   
2     U1081   135085       1            2               1       1   
3     U1001   135085       0            1               1       2   
4     U1056   135085       2            2               2       1   
...     ...      ...     ...          ...             ...     ...   
1156  U1006   132922       2            1               2       1   
1157  U1003   132937       2            2               1       1   
1158  U1027   132937       1            1               1       2   
1159  U1029   132937       1            1               1       1   
1160  U1086   132937       2            0               1       3   

     cl_restaurants  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  
...             ...  
1156              0  
1157              0  
1158              0  
1159              0  
1160              0  

[1161 rows x 7 columns]

In [ ]:
rating.shape

(1161, 5)

In [ ]:
cl_ratings[["rating", "food_rating", "service_rating"]].describe()

rating  food_rating  service_rating
count 1,161.00     1,161.00        1,161.00
mean      1.20         1.22            1.09
std       0.77         0.79            0.79
min       0.00         0.00            0.00
25%       1.00         1.00            0.00
50%       1.00         1.00            1.00
75%       2.00         2.00            2.00
max       2.00         2.00            2.00

In [ ]:
cl_ratings.pivot_table(index = "cl_user", columns="cl_restaurants", values = ["rating", "food_rating", "service_rating"], aggfunc="mean").style.background_gradient(cmap='Blues')

In [ ]:
cl_ratings.groupby("cl_restaurants").mean()

placeID  rating  food_rating  service_rating
cl_restaurants                                                
0              133,925.56    1.17         1.20            1.04
1              134,614.61    1.25         1.25            1.16

In [ ]:
cl_ratings.groupby("cl_user").mean()

placeID  rating  food_rating  service_rating
cl_user                                                
0       133,994.00    0.49         0.52            0.44
1       134,240.67    1.37         1.33            1.24
2       134,180.31    1.23         1.29            1.09
3       134,291.75    1.46         1.58            1.40